### Final Spark Code with Bloom Filter, Differential Privacy, Each exchange minimum price for buy and maximum price for sales filtered every minute

In [1]:
import numpy as np
import hashlib
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, window, struct, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

class SimpleFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    def _hash(self, item, seed):
        hash_val = int(hashlib.sha256(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingWithBloomFilterAndDifferentialPrivacy") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Define the schema of the Kafka message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType()),
    StructField("type", StringType())
])

# Subscribe to Kafka topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Filter for BUY and SELL transactions
buy_df = value_df.filter(col("taker_side") == "BUY")
sell_df = value_df.filter(col("taker_side") == "SELL")

# Function to add Laplacian noise for differential privacy to UUID
def add_noise_to_uuid(uuid):
    noise = np.random.laplace(0, 1.0)
    noisy_uuid = hashlib.sha256((uuid + str(noise)).encode('utf-8')).hexdigest()
    return noisy_uuid

# Group by symbol_id and window time, then aggregate min and max details
buy_price_stats_df = buy_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_min(struct(col("price"), col("uuid"), col("taker_side"), col("symbol_id"), col("time_exchange"))).alias("min_detail")
)
sell_price_stats_df = sell_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_max(struct(col("price"), col("uuid"), col("taker_side"), col("symbol_id"), col("time_exchange"))).alias("max_detail")
)

# Define local path for saving files
local_folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage"

bloom_filter = SimpleFilter(10000, 3)

# Process each batch of data, apply differential privacy to UUID and use Bloom filter
def process_batch(batch_df, batch_id, is_buy):
    pandas_df = batch_df.toPandas()
    detail_column = 'min_detail' if is_buy else 'max_detail'
    # Extract the additional fields
    pandas_df['symbol_id'] = pandas_df[detail_column].apply(lambda x: x['symbol_id'])
    pandas_df['time_exchange'] = pandas_df[detail_column].apply(lambda x: x['time_exchange'])
    pandas_df['price'] = pandas_df[detail_column].apply(lambda x: x['price'])
    pandas_df['uuid'] = pandas_df[detail_column].apply(lambda x: x['uuid']).apply(add_noise_to_uuid)
    pandas_df['taker_side'] = pandas_df[detail_column].apply(lambda x: x['taker_side'])
    
    # Check and add to Bloom filter, save as JSON
    for index, row in pandas_df.iterrows():
        if not bloom_filter.check(row['uuid']):
            bloom_filter.add(row['uuid'])
            json_data = row[['symbol_id', 'time_exchange', 'price', 'uuid', 'taker_side']].to_json()
            file_path = os.path.join(local_folder_path, f"batch_{batch_id}_{index}.json")
            with open(file_path, 'w') as json_file:
                json_file.write(json_data)
            print(f"Row saved to: {file_path}")
            
# Start streaming queries for BUY and SELL batches
buy_query = buy_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, True)).start()
sell_query = sell_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, False)).start()

# Await termination of the queries
buy_query.awaitTermination()
sell_query.awaitTermination()


24/04/27 12:13:31 WARN Utils: Your hostname, Priyas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.79 instead (on interface en0)
24/04/27 12:13:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/priyasuresh/.ivy2/cache
The jars for the packages stored in: /Users/priyasuresh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-64942613-0830-47a2-8648-969ef46f7a0b;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 203ms :: artifacts dl 5ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

Row saved to: /Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage/batch_1_0.json


Row saved to: /Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage/batch_2_0.json


...

Row saved to: /Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage/batch_2296_0.json


Row saved to: /Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage/batch_2274_0.json


ERROR:root:Exception while sending command.age 9160:>               (0 + 0) / 1]
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=80>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/

Py4JError: An error occurred while calling o96.awaitTermination

Row saved to: /Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage/batch_2277_0.json


In [2]:
spark.stop()

24/04/27 14:50:54 WARN StateStore: Error running maintenance thread
java.lang.IllegalStateException: SparkEnv not active, cannot do maintenance on StateStores
	at org.apache.spark.sql.execution.streaming.state.StateStore$.doMaintenance(StateStore.scala:632)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.$anonfun$startMaintenanceIfNeeded$1(StateStore.scala:610)
	at org.apache.spark.sql.execution.streaming.state.StateStore$MaintenanceTask$$anon$1.run(StateStore.scala:453)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.

In [ ]:
#Below code works well and store results in local folder--above code trying to fix code issues for json file format

In [37]:
import numpy as np
import hashlib
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, window, struct, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

class SimpleFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    def _hash(self, item, seed):
        hash_val = int(hashlib.sha256(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingWithBloomFilterAndDifferentialPrivacy") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Define the schema of the Kafka message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType()),
    StructField("type", StringType())
])

# Subscribe to Kafka topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Filter for BUY and SELL transactions
buy_df = value_df.filter(col("taker_side") == "BUY")
sell_df = value_df.filter(col("taker_side") == "SELL")

# Function to add Laplacian noise for differential privacy to UUID
def add_noise_to_uuid(uuid):
    noise = np.random.laplace(0, 1.0)
    noisy_uuid = hashlib.sha256((uuid + str(noise)).encode('utf-8')).hexdigest()
    return noisy_uuid

# Group by symbol_id and window time, then aggregate min and max details
buy_price_stats_df = buy_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_min(struct(col("price"), col("uuid"), col("taker_side"), col("symbol_id"), col("time_exchange"))).alias("min_detail")
)
sell_price_stats_df = sell_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_max(struct(col("price"), col("uuid"), col("taker_side"), col("symbol_id"), col("time_exchange"))).alias("max_detail")
)

# Define local path for saving files
local_folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage"

bloom_filter = SimpleFilter(10000, 3)

# Process each batch of data, apply differential privacy to UUID and use Bloom filter
def process_batch(batch_df, batch_id, is_buy):
    pandas_df = batch_df.toPandas()
    detail_column = 'min_detail' if is_buy else 'max_detail'
    # Extract the additional fields
    pandas_df['symbol_id'] = pandas_df[detail_column].apply(lambda x: x['symbol_id'])
    pandas_df['time_exchange'] = pandas_df[detail_column].apply(lambda x: x['time_exchange'])
    pandas_df['price'] = pandas_df[detail_column].apply(lambda x: x['price'])
    pandas_df['uuid'] = pandas_df[detail_column].apply(lambda x: x['uuid']).apply(add_noise_to_uuid)
    pandas_df['taker_side'] = pandas_df[detail_column].apply(lambda x: x['taker_side'])
    # Check and add to Bloom filter
    for index, row in pandas_df.iterrows():
        if not bloom_filter.check(row['uuid']):
            bloom_filter.add(row['uuid'])
            # Save to CSV with additional fields
            file_path = os.path.join(local_folder_path, f"batch_{batch_id}_{index}.csv")
            row[['symbol_id', 'time_exchange', 'price', 'uuid', 'taker_side']].to_csv(file_path, index=False)
            print(f"Row saved to: {file_path}")
            
# Start streaming queries for BUY and SELL batches
buy_query = buy_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, True)).start()
sell_query = sell_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, False)).start()

# Await termination of the queries
buy_query.awaitTermination()
sell_query.awaitTermination()


24/04/20 22:26:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-6a354a21-ddbf-402b-93b1-0760a7a3a386. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/20 22:26:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/20 22:26:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-18f6c6d2-8cda-45f3-84b7-8611aaff4a73. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folde

Py4JError: An error occurred while calling o21643.awaitTermination

In [38]:
spark.stop()

24/04/20 22:42:55 WARN StateStore: Error running maintenance thread
java.lang.IllegalStateException: SparkEnv not active, cannot do maintenance on StateStores
	at org.apache.spark.sql.execution.streaming.state.StateStore$.doMaintenance(StateStore.scala:632)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.$anonfun$startMaintenanceIfNeeded$1(StateStore.scala:610)
	at org.apache.spark.sql.execution.streaming.state.StateStore$MaintenanceTask$$anon$1.run(StateStore.scala:453)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.

In [27]:
import numpy as np
import hashlib
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, window, struct, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
import os

class SimpleFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    def _hash(self, item, seed):
        hash_val = int(hashlib.sha256(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingWithBloomFilterAndDifferentialPrivacy") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Define the schema of the Kafka message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType()),
    StructField("type", StringType())
])

# Subscribe to Kafka topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Filter for BUY and SELL transactions
buy_df = value_df.filter(col("taker_side") == "BUY")
sell_df = value_df.filter(col("taker_side") == "SELL")

# Function to add Laplacian noise for differential privacy to UUID
def add_noise_to_uuid(uuid):
    noise = np.random.laplace(0, 1.0)
    noisy_uuid = hashlib.sha256((uuid + str(noise)).encode('utf-8')).hexdigest()
    return noisy_uuid

# Group by symbol_id and window time, then aggregate min and max details
buy_price_stats_df = buy_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_min(struct(col("price"), col("uuid"), col("taker_side"))).alias("min_detail")
)
sell_price_stats_df = sell_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_max(struct(col("price"), col("uuid"), col("taker_side"))).alias("max_detail")
)

# Define local path for saving files
local_folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/intermediate_storage"

bloom_filter = SimpleFilter(10000, 3)

# Process each batch of data, apply differential privacy to UUID and use Bloom filter
def process_batch(batch_df, batch_id, is_buy):
    pandas_df = batch_df.toPandas()
    detail_column = 'min_detail' if is_buy else 'max_detail'
    pandas_df['price'] = pandas_df[detail_column].apply(lambda x: x['price'])
    pandas_df['uuid'] = pandas_df[detail_column].apply(lambda x: x['uuid']).apply(add_noise_to_uuid)
    pandas_df['taker_side'] = pandas_df[detail_column].apply(lambda x: x['taker_side'])
    for index, row in pandas_df.iterrows():
        if not bloom_filter.check(row['uuid']):
            bloom_filter.add(row['uuid'])
            file_path = os.path.join(local_folder_path, f"batch_{batch_id}_{index}.csv")
            row.to_csv(file_path, index=False)
            print(f"Row saved to: {file_path}")

# Start streaming queries for BUY and SELL batches
buy_query = buy_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, True)).start()
sell_query = sell_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, False)).start()

# Await termination of the queries
buy_query.awaitTermination()
sell_query.awaitTermination()


24/04/20 19:47:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-3a1e1c11-1560-42f9-b33a-dcc6e8d9e312. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/20 19:47:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/20 19:47:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-59f6c961-0030-43bc-8296-469c06ac6b2c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folde

Py4JError: An error occurred while calling o11309.awaitTermination

In [36]:
spark.stop()

24/04/20 22:25:32 WARN StateStore: Error running maintenance thread
java.lang.IllegalStateException: SparkEnv not active, cannot do maintenance on StateStores
	at org.apache.spark.sql.execution.streaming.state.StateStore$.doMaintenance(StateStore.scala:632)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.$anonfun$startMaintenanceIfNeeded$1(StateStore.scala:610)
	at org.apache.spark.sql.execution.streaming.state.StateStore$MaintenanceTask$$anon$1.run(StateStore.scala:453)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.

In [ ]:
Below code has GCP storage code info

In [24]:
import numpy as np
import hashlib
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, window, struct, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
import os
from google.cloud import storage

class SimpleFilter:
    # Initialize the Bloom Filter with the given size and hash count
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    # Create a hash of the item using SHA-256 and a seed value
    def _hash(self, item, seed):
        hash_val = int(hashlib.sha256(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    # Add an item to the Bloom Filter
    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    # Check if an item is possibly in the Bloom Filter
    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingWithBloomFilterAndDifferentialPrivacy") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Define the schema of the Kafka message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType()),
    StructField("type", StringType())
])

# Subscribe to Kafka topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Filter for BUY and SELL transactions
buy_df = value_df.filter(col("taker_side") == "BUY")
sell_df = value_df.filter(col("taker_side") == "SELL")

# Function to add Laplacian noise for differential privacy to UUID
def add_noise_to_uuid(uuid):
    noise = np.random.laplace(0, 1.0)
    noisy_uuid = hashlib.sha256((uuid + str(noise)).encode('utf-8')).hexdigest()
    return noisy_uuid


# Function to add Laplacian noise for differential privacy
def add_noise(data):
    return data + np.random.laplace(0, 1.0)

# Group by symbol_id and window time, then aggregate min and max details
buy_price_stats_df = buy_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_min(struct(col("price"), col("uuid"), col("taker_side"))).alias("min_detail")
)
sell_price_stats_df = sell_df.groupBy("symbol_id", window("time_exchange", "1 minute")).agg(
    spark_max(struct(col("price"), col("uuid"), col("taker_side"))).alias("max_detail")
)

# Define local path and GCS bucket for saving files
local_folder_path = "/Users/priyasuresh/googlecloud"
bucket_name = "bigdataproject_cryptocurrency"

bloom_filter = SimpleFilter(10000, 3)

# Process each batch of data, apply differential privacy to UUID and use Bloom filter
def process_batch(batch_df, batch_id, is_buy):
    # Convert batch to Pandas DataFrame for processing
    pandas_df = batch_df.toPandas()
    # Extract details from the struct
    detail_column = 'min_detail' if is_buy else 'max_detail'
    pandas_df['price'] = pandas_df[detail_column].apply(lambda x: x['price'])
    pandas_df['uuid'] = pandas_df[detail_column].apply(lambda x: x['uuid']).apply(add_noise_to_uuid)
    pandas_df['taker_side'] = pandas_df[detail_column].apply(lambda x: x['taker_side'])
    # Check and add to Bloom filter
    for index, row in pandas_df.iterrows():
        if not bloom_filter.check(row['uuid']):
            bloom_filter.add(row['uuid'])
            # Save to CSV, upload to GCS, then delete local file
            file_path = os.path.join(local_folder_path, f"batch_{batch_id}_{index}.csv")
            row.to_csv(file_path, index=False)
            print(f"Row saved to: {file_path}")
            upload_to_gcs(file_path)
            os.remove(file_path)

def upload_to_gcs(file_path):
    # Upload file to Google Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    destination_blob_name = os.path.basename(file_path)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(file_path)
    print(f"File uploaded to GCS: gs://{bucket_name}/{destination_blob_name}")

# Start streaming queries for BUY and SELL batches
buy_query = buy_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, True)).start()
sell_query = sell_price_stats_df.writeStream.outputMode("update").foreachBatch(lambda df, epoch_id: process_batch(df, epoch_id, False)).start()

# Await termination of the queries
buy_query.awaitTermination()
sell_query.awaitTermination()


24/04/20 14:39:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-544c693e-94e2-44e7-9763-63be7982162f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/20 14:39:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/20 14:39:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-2b12f70e-7eed-4b43-9e35-35b168cd30a2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folde

Py4JError: An error occurred while calling o7789.awaitTermination

In [26]:
spark.stop()

24/04/20 15:02:50 WARN StateStore: Error running maintenance thread
java.lang.IllegalStateException: SparkEnv not active, cannot do maintenance on StateStores
	at org.apache.spark.sql.execution.streaming.state.StateStore$.doMaintenance(StateStore.scala:632)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.$anonfun$startMaintenanceIfNeeded$1(StateStore.scala:610)
	at org.apache.spark.sql.execution.streaming.state.StateStore$MaintenanceTask$$anon$1.run(StateStore.scala:453)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.